In [1]:
!pip install nltk textblob
!pip install torch transformers
!pip install happytransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.metrics import f1_score, confusion_matrix
import string
import textwrap
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### EDA and Cleaning

In [4]:
df = pd.read_csv('/content/drive/My Drive/datasets/JobBiasDataset/final_dataset_task3.csv')#,nrows=20)
df1_lower = df.copy()
df1_lower['Job_description'] = df['Job_description'].str.lower()

df1_lower.head()

,Unnamed: 0,Job_description,Length,Lemmatized_description,male_bias,female_bias,racial_bias,% male_bias,% female_bias,% racial_bias,male_bias_yes,female_bias_yes,racial_bias_yes,salary,numerical_salary,gender_bias,ratio_female_male_bias
0,0,smb account executive identify qualify new opp...,303,"['smb', 'account', 'executive', 'identify', 'q...",6,2,0,1.980198,0.660066,0.000000,1,1,0,NaN,0.0,Male,0.333333
1,1,looking key new member join small talented pro...,246,"['looking', 'key', 'new', 'member', 'join', 's...",6,2,0,2.439024,0.813008,0.000000,1,1,0,NaN,0.0,Male,0.333333
2,2,looking data analyst software developer work r...,144,"['looking', 'data', 'analyst', 'software', 'de...",0,1,1,0.000000,0.694444,0.694444,0,1,1,NaN,0.0,Female,0.000000
3,3,creative virtual therefore seeking energetic e...,149,"['creative', 'virtual', 'therefore', 'seeking'...",1,3,0,0.671141,2.013423,0.000000,1,1,0,NaN,0.0,Female,3.000000
4,4,essential skills minimum bachelor degree equiv...,94,"['essential', 'skill', 'minimum', 'bachelor', ...",0,0,0,0.000000,0.000000,0.000000,0,0,0,NaN,0.0,Female,0.000000


In [5]:
duplicates_per_column = df1_lower.apply(lambda x: x.duplicated().sum())
print(duplicates_per_column)

Unnamed: 0                    0
Job_description              72
Length                    22311
Lemmatized_description      846
male_bias                 23337
female_bias               23320
racial_bias               23360
% male_bias               19971
% female_bias             19206
% racial_bias             22347
male_bias_yes             23370
female_bias_yes           23370
racial_bias_yes           23370
salary                    23137
numerical_salary          23212
gender_bias               23370
ratio_female_male_bias    23052
dtype: int64


In [6]:
df1_lower = df1_lower.drop_duplicates( subset =['Job_description'], keep ='first')

In [7]:
df1_lower = df1_lower[['Job_description']].copy()
df1_lower.head()

,Job_description
0,smb account executive identify qualify new opp...
1,looking key new member join small talented pro...
2,looking data analyst software developer work r...
3,creative virtual therefore seeking energetic e...
4,essential skills minimum bachelor degree equiv...


In [8]:
def remove_html_tags(text):
    if isinstance(text, str):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)
    else:
        return text

columns_to_clean = ['Job_description']

for column in columns_to_clean:
    df[column] = df[column].apply(remove_html_tags)


In [9]:
def preprocess(q):

    q=str(q).lower().strip()
    q = q.replace("â€™","'") # added by Saurabh
    q = q.replace("job description: ","")
    q = q.replace("job description ","")
    q = q.replace("job description","")
    text = '''
    Job Details


    Job Location
    Saudi Arabia


    Company Industry
    Other Business Support Services


    Company Type
    Unspecified


    Employment Type
    Unspecified


    Monthly Salary Range
    Unspecified


    Number of Vacancies
    Unspecified
    '''
    q = q.replace(text,"")

    q = q.replace("Â","")
    q = q.replace("Ã","")

    q = q.replace("$","dollar")
    q = q.replace("₹","rupee")
    q = q.replace("%","percent")
    q = q.replace("@","at")
    q = q.replace("€","euro")

    q = q.replace('[math]', '')

    q = q.replace(', 000,000,000', 'b ')
    q = q.replace(', 000,000', 'm ')
    q = q.replace(', 000', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)


    #decontracting words
    contractions = {
    "ain't": "am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]
        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ill", " will")

    #removing html tags
    q = BeautifulSoup(q)
    q = q.get_text()

    #removing punctuations
    pattern = re.compile('\W')

    q=re.sub(pattern , ' ', q).strip()

    return q

# preprocess
df1_lower['Job_description'] = df1_lower['Job_description'].apply(preprocess)
# remove extra spaces
df1_lower['Job_description'] = df1_lower['Job_description'].str.strip().str.replace(r'\s+', ' ', regex=True)

In [10]:
masculine_words = ['ambition', 'driven', 'lead', 'persist', 'principle', 'decision', 'superior', 'individual', 'assertive', 'strong', 'hierarchical', 'rigid', 'silicon valley', 'stock options', 'takes risk', 'workforce', 'autonomous', 'ping pong', 'pool table', 'must', 'competitive', 'he', 'his', 'himself', 'confident', 'active', 'aggressive', 'ambitious', 'fearless', 'headstrong', 'defensive', 'independent', 'dominant', 'outspoken', 'leader', 'fast paced', 'adventurous', 'analytical', 'decisive', 'determined', 'ninja', 'objective', 'rock star', 'boast', 'challenging', 'courage']
feminine_words = ['thoughtful', 'creative', 'adaptable', 'choose', 'curious', 'excellent', 'flexible', 'multitasking', 'health', 'imaginative', 'intuitive', 'leans in', 'plans for the future', 'resilient', 'self-aware', 'socially responsible', 'trustworthy', 'shup-to-date', 'wellness program', 'nurture', 'teach', 'dependable', 'community', 'serving', 'loyal', 'enthusiasm', 'interpersonal', 'connect', 'commit', 'she', 'agree', 'empathy', 'sensitive', 'affectionate', 'feel', 'support', 'collaborate', 'honest', 'trust', 'understand', 'comapssion', 'share', 'polite', 'kind', 'caring', 'her', 'hers', 'herself', 'feminine', 'cheer', 'communal', 'emotional', 'flatterable', 'gentle', 'interdependent', 'kinship', 'modesty', 'pleasant', 'polite', 'quiet', 'sympathy', 'warm', 'dominant', 'yield']
racial_words = ['native english speaker', 'professionally groomed hair', 'native', 'culture fit', 'non-white', 'clean-shaven', 'neat hairstyle', 'master', 'slave', 'a cakewalk', 'brownbag session', 'spirit animal', 'digital native', 'servant leadership', 'tribe', 'oriental', 'spic', 'english fluency', 'level native', 'illegals', 'eskimo', 'latino', 'latina', 'migrant', 'blacklist', 'whitelist']


In [11]:
df_final = df1_lower.copy()
df_final.head(2)


,Job_description
0,smb account executive identify qualify new opp...
1,looking key new member join small talented pro...


In [12]:
df_final.shape

(23300, 1)

In [13]:
job_descriptions = df_final['Job_description']

def analyze_job_description(job_description):
    tokens = word_tokenize(job_description.lower())

    # Analyze gender (example: finding gendered pronouns)
    gendered_pronouns = [
          'ambition', 'driven', 'lead', 'persist', 'principle', 'decision', 'superior', 'individual', 'assertive', 'strong', 'hierarchical', 'rigid', 'silicon valley', 'stock options', 'takes risk', 'workforce', 'autonomous', 'ping pong', 'pool table', 'must', 'competitive', 'he', 'his', 'himself', 'confident', 'active', 'aggressive', 'ambitious', 'fearless', 'headstrong', 'defensive', 'independent', 'dominant', 'outspoken', 'leader', 'fast paced', 'adventurous', 'analytical', 'decisive', 'determined', 'ninja', 'objective', 'rock star', 'boast', 'challenging', 'courage',
          'thoughtful', 'creative', 'adaptable', 'choose', 'curious', 'excellent', 'flexible', 'multitasking', 'health', 'imaginative', 'intuitive', 'leans in', 'plans for the future', 'resilient', 'self-aware', 'socially responsible', 'trustworthy', 'shup-to-date', 'wellness program', 'nurture', 'teach', 'dependable', 'community', 'serving', 'loyal', 'enthusiasm', 'interpersonal', 'connect', 'commit', 'she', 'agree', 'empathy', 'sensitive', 'affectionate', 'feel', 'support', 'collaborate', 'honest', 'trust', 'understand', 'compassion', 'share', 'polite', 'kind', 'caring', 'her', 'hers', 'herself', 'feminine', 'cheer', 'communal', 'emotional', 'flatterable', 'gentle', 'interdependent', 'kinship', 'modesty', 'pleasant', 'polite', 'quiet', 'sympathy', 'warm', 'dominant', 'yield',
          'native english speaker', 'professionally groomed hair', 'native', 'culture fit', 'non-white', 'clean-shaven', 'neat hairstyle', 'master', 'slave', 'a cakewalk', 'brownbag session', 'spirit animal', 'digital native', 'servant leadership', 'tribe', 'oriental', 'spic', 'english fluency', 'level native', 'illegals', 'eskimo', 'latino', 'latina', 'migrant', 'blacklist', 'whitelist'
      ]
    gendered_words = [word for word in tokens if word in gendered_pronouns]
    gendered_word_count = len(gendered_words)
    if gendered_word_count > 0:
      label = 1
    else:
      label = 0

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    word_freq = FreqDist(filtered_tokens)
    blob = TextBlob(job_description)
    sentiment = blob.sentiment
    sentences = sent_tokenize(job_description)
    sentence_count = len(sentences)

    # Analyze tone (positive 1, negative 0, neutral 0.5)
    if sentiment.polarity > 0:
        tone = 1
    elif sentiment.polarity < 0:
        tone = 0
    else:
        tone = 0.5

    '''
    return {
        "Number of Sentences": sentence_count,
        "Sentiment": tone,
        "Polarity": sentiment.polarity,
        "Subjectivity": sentiment.subjectivity,
        "Word Frequency": word_freq.most_common(10)
    }
    '''
    features = [0.0]*6

    features[0] = sentence_count
    features[1] = tone
    features[2] = sentiment.polarity
    features[3] = sentiment.subjectivity
    features[4] = gendered_word_count
    features[5] = label
    return features


analysis_results = df_final['Job_description'].apply(analyze_job_description)

df_final['sentence_count'] = list(map(lambda x: x[0], analysis_results))
df_final['tone'] = list(map(lambda x: x[1], analysis_results))
df_final['polarity'] = list(map(lambda x: x[2], analysis_results))
df_final['subjectivity'] = list(map(lambda x: x[3], analysis_results))
df_final['gendered_word_count'] = list(map(lambda x: x[4], analysis_results))
df_final['label'] = list(map(lambda x: x[5], analysis_results))

In [14]:
df_final = df_final.dropna(subset=['Job_description'])
df_final.shape

(23300, 7)

In [15]:
df_final.head()

,Job_description,sentence_count,tone,polarity,subjectivity,gendered_word_count,label
0,smb account executive identify qualify new opp...,1,1.0,0.210238,0.487290,6,1
1,looking key new member join small talented pro...,1,1.0,0.257867,0.589405,7,1
2,looking data analyst software developer work r...,1,1.0,0.221460,0.443554,3,1
3,creative virtual therefore seeking energetic e...,1,1.0,0.178869,0.425967,5,1
4,essential skills minimum bachelor degree equiv...,1,1.0,0.236667,0.383333,0,0


### Model Creation

In [16]:
# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_final['Job_description'])
vocab_size = len(tokenizer.word_index) + 1
max_length = max([len(word_tokenize(text)) for text in df_final['Job_description']])
X = tokenizer.texts_to_sequences(df_final['Job_description'])
X = pad_sequences(X, maxlen=max_length, padding='post')

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, df_final['label'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the sizes of the datasets
print(f"Training data size: {len(X_train)}")
print(f"Validation data size: {len(X_val)}")
print(f"Test data size: {len(X_test)}")

Training data size: 18640
Validation data size: 2330
Test data size: 2330


In [17]:

# Build the LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=4)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/5
4660/4660 [==============================] - 514s 109ms/step - loss: 0.2679 - accuracy: 0.9254
Epoch 2/5
4660/4660 [==============================] - 432s 93ms/step - loss: 0.2662 - accuracy: 0.9256
Epoch 3/5
4660/4660 [==============================] - 427s 92ms/step - loss: 0.2659 - accuracy: 0.9256
Epoch 4/5
4660/4660 [==============================] - 420s 90ms/step - loss: 0.2659 - accuracy: 0.9256
Epoch 5/5
73/73 [==============================] - 3s 32ms/step - loss: 0.2720 - accuracy: 0.9236
Test Accuracy: 92.36%


In [18]:
model.save('/content/drive/My Drive/datasets/JobBiasDataset/lstm_model')

In [19]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict labels on the test data
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# Calculate the confusion matrix
confusion = confusion_matrix(y_test, y_pred_binary)

# Extract TP, TN, FP, FN
TN, FP, FN, TP = confusion.ravel()

# Calculate precision, recall, and F1-score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the confusion matrix and performance metrics
print("Confusion Matrix:")
print(confusion)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_binary, target_names=['Neutral', 'Biased']))
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")


73/73 [==============================] - 4s 40ms/step
Confusion Matrix:
[[   0  178]
 [   0 2152]]

Classification Report:
              precision    recall  f1-score   support

     Neutral       0.00      0.00      0.00       178
      Biased       0.92      1.00      0.96      2152

    accuracy                           0.92      2330
   macro avg       0.46      0.50      0.48      2330
weighted avg       0.85      0.92      0.89      2330

Precision: 0.92
Recall: 1.00
F1-Score: 0.96


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict labels on the test data
y_pred = model.predict(X_val)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# Calculate the confusion matrix
confusion = confusion_matrix(y_test, y_pred_binary)

# Extract TP, TN, FP, FN
TN, FP, FN, TP = confusion.ravel()

# Calculate precision, recall, and F1-score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the confusion matrix and performance metrics
print("Confusion Matrix:")
print(confusion)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_binary, target_names=['Neutral', 'Biased']))
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")


73/73 [==============================] - 2s 31ms/step
Confusion Matrix:
[[   0  178]
 [   0 2152]]

Classification Report:
              precision    recall  f1-score   support

     Neutral       0.00      0.00      0.00       178
      Biased       0.92      1.00      0.96      2152

    accuracy                           0.92      2330
   macro avg       0.46      0.50      0.48      2330
weighted avg       0.85      0.92      0.89      2330

Precision: 0.92
Recall: 1.00
F1-Score: 0.96


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Generate new BIASED Free Description

In [21]:
gendered_pronouns = [
          'ambition', 'driven', 'lead', 'persist', 'principle', 'decision', 'superior', 'individual', 'assertive',
          'strong', 'hierarchical', 'rigid', 'silicon valley', 'stock options', 'takes risk', 'workforce', 'autonomous',
          'ping pong', 'pool table', 'must', 'competitive', 'he', 'his', 'himself', 'confident', 'active', 'aggressive',
          'ambitious', 'fearless', 'headstrong', 'defensive', 'independent', 'dominant', 'outspoken', 'leader', 'fast paced',
          'adventurous', 'analytical', 'decisive', 'determined', 'ninja', 'objective', 'rock star', 'boast', 'challenging', 'courage',
          'thoughtful', 'creative', 'adaptable', 'choose', 'curious', 'excellent', 'flexible', 'multitasking', 'health',
          'imaginative', 'intuitive', 'leans in', 'plans for the future', 'resilient', 'self-aware', 'socially responsible',
          'trustworthy', 'shup-to-date', 'wellness program', 'nurture', 'teach', 'dependable', 'community', 'serving', 'loyal',
          'enthusiasm', 'interpersonal', 'connect', 'commit', 'she', 'agree', 'empathy', 'sensitive', 'affectionate', 'feel',
          'support', 'collaborate', 'honest', 'trust', 'understand', 'compassion', 'share', 'polite', 'kind', 'caring', 'her',
          'hers', 'herself', 'feminine', 'cheer', 'communal', 'emotional', 'flatterable', 'gentle', 'interdependent', 'kinship',
          'modesty', 'pleasant', 'polite', 'quiet', 'sympathy', 'warm', 'dominant', 'yield',
          'native english speaker', 'professionally groomed hair', 'native', 'culture fit', 'non-white', 'clean-shaven',
          'neat hairstyle', 'master', 'slave', 'a cakewalk', 'brownbag session', 'spirit animal', 'digital native',
          'servant leadership', 'tribe', 'oriental', 'spic', 'english fluency', 'level native', 'illegals', 'eskimo',
          'latino', 'latina', 'migrant', 'blacklist', 'whitelist'
      ]

# List of neutral words
neutral_words = [
    'ambition', 'driven', 'lead', 'persist', 'principle', 'decision', 'superior', 'individual', 'assertive', 'strong',
    'hierarchical', 'rigid', 'silicon valley', 'stock options', 'takes risk', 'workforce', 'autonomous', 'ping pong',
    'pool table', 'must', 'competitive', 'he', 'his', 'himself', 'confident', 'active', 'aggressive', 'ambitious',
    'fearless', 'headstrong', 'defensive', 'independent', 'dominant', 'outspoken', 'leader', 'fast paced', 'adventurous',
    'analytical', 'decisive', 'determined', 'ninja', 'objective', 'rock star', 'boast', 'challenging', 'courage',
    'thoughtful', 'creative', 'adaptable', 'choose', 'curious', 'excellent', 'flexible', 'multitasking', 'health',
    'imaginative', 'intuitive', 'leans in', 'plans for the future', 'resilient', 'self-aware', 'socially responsible',
    'trustworthy', 'shup-to-date', 'wellness program', 'nurture', 'teach', 'dependable', 'community', 'serving', 'loyal',
    'enthusiasm', 'interpersonal', 'connect', 'commit', 'she', 'agree', 'empathy', 'sensitive', 'affectionate', 'feel',
    'support', 'collaborate', 'honest', 'trust', 'understand', 'compassion', 'share', 'polite', 'kind', 'caring', 'her',
    'hers', 'herself', 'feminine', 'cheer', 'communal', 'emotional', 'flatterable', 'gentle', 'interdependent', 'kinship',
    'modesty', 'pleasant', 'polite', 'quiet', 'sympathy', 'warm', 'dominant', 'yield',
    'native english speaker', 'professionally groomed hair', 'native', 'culture fit', 'non-white', 'clean-shaven',
    'neat hairstyle', 'master', 'slave', 'a cakewalk', 'brownbag session', 'spirit animal', 'digital native',
    'servant leadership', 'tribe', 'oriental', 'spic', 'english fluency', 'level native', 'illegals', 'eskimo', 'latino',
    'latina', 'migrant', 'blacklist', 'whitelist'
]



word_dict = dict(zip(gendered_pronouns, neutral_words))

# Print the dictionary
for key, value in word_dict.items():
    print(f"{key}: {value}")


ambition: ambition
driven: driven
lead: lead
persist: persist
principle: principle
decision: decision
superior: superior
individual: individual
assertive: assertive
strong: strong
hierarchical: hierarchical
rigid: rigid
silicon valley: silicon valley
stock options: stock options
takes risk: takes risk
workforce: workforce
autonomous: autonomous
ping pong: ping pong
pool table: pool table
must: must
competitive: competitive
he: he
his: his
himself: himself
confident: confident
active: active
aggressive: aggressive
ambitious: ambitious
fearless: fearless
headstrong: headstrong
defensive: defensive
independent: independent
dominant: dominant
outspoken: outspoken
leader: leader
fast paced: fast paced
adventurous: adventurous
analytical: analytical
decisive: decisive
determined: determined
ninja: ninja
objective: objective
rock star: rock star
boast: boast
challenging: challenging
courage: courage
thoughtful: thoughtful
creative: creative
adaptable: adaptable
choose: choose
curious: curious

In [22]:
from happytransformer import HappyTextToText, TTSettings

In [23]:
import re

def replace_gendered_pronouns(text):
    # Use regular expressions to find and replace gendered pronouns in the text
    for pronoun, replacement in word_dict.items():
        # Use word boundaries to match whole words only
        pattern = r'\b' + re.escape(pronoun) + r'\b'
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

    return text

# Example usage:
original_sentence = "He is a diligent worker, and his dedication is commendable."
rewritten_sentence = replace_gendered_pronouns(original_sentence)
print(rewritten_sentence)



happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
args = TTSettings(num_beams=5, min_length=1)
# Add the prefix "grammar: " before each input
result = happy_tt.generate_text("grammar: "+rewritten_sentence, args=args)
print(result.text) # This sentence has bad grammar.

he is a diligent worker, and his dedication is commendable.


He is a diligent worker, and his dedication is commendable.


In [24]:
'''

df = pd.read_csv('/content/drive/My Drive/datasets/JobBiasDataset/DataSet.csv')
df['description'] = df['description'] + df['requirements']
df.drop(['requirements'], axis=1, inplace=True)
df = df.dropna()

def remove_html_tags(text):
    if isinstance(text, str):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)
    else:
        return text

columns_to_clean = ['description']

for column in columns_to_clean:
    df[column] = df[column].apply(remove_html_tags)

job_descriptions = df['description']

def analyze_job_description(job_description):
    tokens = word_tokenize(job_description.lower())

    # Analyze gender (example: finding gendered pronouns)
    gendered_pronouns = ["he", "him", "his", "she", "her", "hers", 'agree', 'affectionate', 'careful', 'child',
                        'cheer', 'collab', 'commit', 'communal', 'compassion', 'conscientious', 'connect',
                        'considerate', 'cooperat', 'co-operat', 'dedicated', 'depend', 'diligent', 'emotiona', 'empath',
                        'feel', 'flatterable', 'gentle', 'hardwork', 'honest', 'interpersonal', 'interdependen',
                        'interpersona', 'inter-personal', 'inter-dependen', 'inter-persona', 'kind', 'kinship',
                        'loyal', 'meticulous', 'modesty', 'nag', 'nurtur', 'pleasant', 'polite', 'quiet', 'respon',
                        'sensitiv', 'submissive', 'support', 'sympath', 'tender', 'thorough', 'together', 'trust',
                        'understand', 'warm', 'whin', 'enthusias', 'inclusive', 'yield', "male", "female", "females",
                        'ladylike', 'girl', 'boy', 'gents', 'lady', 'ladies', 'active', 'adventurous', 'aggress',
                        'ambitio', 'analy', 'assert', 'athlet', 'autonom', 'battle', 'boast', 'challeng', 'champion',
                        'compet', 'confident', 'courag', 'decid', 'decision', 'decisive', 'defend', 'determin', 'domina',
                        'dominant', 'driven', 'fearless', 'fight', 'force', 'greedy', 'head-strong', 'headstrong',
                        'hierarch', 'hostil', 'impulsive', 'independen', 'individual', 'intellect', 'lead', 'logic',
                        'objective', 'opinion', 'outspoken', 'persist', 'principle', 'reckless', 'self-confiden',
                        'self-relian', 'self-sufficien', 'selfconfiden', 'selfrelian', 'selfsufficien', 'stubborn',
                        'superior', 'unreasonab', 'rockstar', 'guru', 'fast-paced', 'girls']
    gendered_words = [word for word in tokens if word in gendered_pronouns]
    gendered_word_count = len(gendered_words)
    if gendered_word_count > 0:
        label = 1
    else:
        label = 0

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    word_freq = FreqDist(filtered_tokens)
    blob = TextBlob(job_description)
    sentiment = blob.sentiment
    sentences = sent_tokenize(job_description)
    sentence_count = len(sentences)

    # Analyze tone (positive 1, negative 0, neutral 0.5)
    if sentiment.polarity > 0:
        tone = 1
    elif sentiment.polarity < 0:
        tone = 0
    else:
        tone = 0.5

    return {
        "Number of Sentences": sentence_count,
        "Sentiment": tone,
        "Polarity": sentiment.polarity,
        "Subjectivity": sentiment.subjectivity,
        "Word Frequency": word_freq.most_common(10),
        "label":label
    }

analysis_results = df['description'].apply(analyze_job_description)

df['sentence_count'] = list(map(lambda x: x["Number of Sentences"], analysis_results))
df['tone'] = list(map(lambda x: x["Sentiment"], analysis_results))
df['polarity'] = list(map(lambda x: x["Polarity"], analysis_results))
df['subjectivity'] = list(map(lambda x: x["Subjectivity"], analysis_results))
df['gendered_word_count'] = list(map(lambda x: x["Word Frequency"], analysis_results))
df['label'] = list(map(lambda x: x["label"], analysis_results))

'''

'\n\ndf = pd.read_csv(\'/content/drive/My Drive/datasets/JobBiasDataset/DataSet.csv\')\ndf[\'description\'] = df[\'description\'] + df[\'requirements\']\ndf.drop([\'requirements\'], axis=1, inplace=True)\ndf = df.dropna()\n\ndef remove_html_tags(text):\n    if isinstance(text, str):\n        clean = re.compile(\'<.*?>\')\n        return re.sub(clean, \'\', text)\n    else:\n        return text\n\ncolumns_to_clean = [\'description\']\n\nfor column in columns_to_clean:\n    df[column] = df[column].apply(remove_html_tags)\n\njob_descriptions = df[\'description\']\n\ndef analyze_job_description(job_description):\n    tokens = word_tokenize(job_description.lower())\n\n    # Analyze gender (example: finding gendered pronouns)\n    gendered_pronouns = ["he", "him", "his", "she", "her", "hers", \'agree\', \'affectionate\', \'careful\', \'child\',\n                        \'cheer\', \'collab\', \'commit\', \'communal\', \'compassion\', \'conscientious\', \'connect\',\n                        \'